# Bollinger Bands

* It's capable of measuring the **volatility** of the prices
* It helps to visualize maximums and minimums 
* It helps to identify **overvaluation** and **undervaluation** of the prices
  * When the price is **overvalued** you may want to **sell it** expecting that after that appreciation period, the prices will **decrease**
  * When the price is **devalued** you may want to **buy it** expecting that after that depreciation period, the prices will **increase**




* The upper and lower bands creates a kind of channel where the stock prices lives in



The upper band is calculated by:
$$
upperband = sma( \ lastdays(20) \ ) + ( \ 2 * sdev(lastdays(20)) \ )
$$


The lower band is calculated by:
$$
upperband = sma( \ lastdays(20) \ ) - ( \ 2 * sdev(lastdays(20)) \ )
$$


* $20$ is the number of contiguous days considered to calculate the standard deviation and the moving average - normally it's treated as a standard.
* The standard deviation is multiplied by $2$ using a statistics fact that in a normal distribution $95\%$ of the values will be in a distance of $2 * stdev$ from the average, but this is not totally true for this case because the prices may not follow a normal distribution



## Support and resistance

* Buyers and sellers can take control of the market
  * If everyone wants to buy, by the low of supply and demand, the prices shall raise
  * In this case, a depreciation may be reverted, and the prices begin to raise
  * In the analogous case, if everyone wants to sell, the price will decrease
  * With more product in the market, the price will tend to fall



* **Support** is a region where the prices face a difficulty to continue to **fall** e.g. **buyers** are taking control of the market 
* **Resistance** is a region where the prices face a difficulty to continue to **raise** e.g. **sellers** are taking control of the market 



![](https://i.ibb.co/GRxBd4v/Screenshot-20200213-111254.png)



## Channel break

* When the lines of the channel become closer and a well defined channel is formed - the supply and demand may be in equilibrium 

* When the prices broke of the channel there may be a strong tendency of raise or fall of the prices



**But** a channel break does not mean that you should buy or sell anything - it's necessary to use other trading indicators to confirm the decision



## Bollinger laws

1. The Bollinger bands provide a relative definition of cheap and expensive - by definition **upper band** means *expensive* and **lower band** means *cheap*
2. The relative definition can be applied to compare the price movement but, the movement itself (not only the bands) should be used to make more precise buy and sell decisions
3. Useful trading indicators can be derived from
   1. momentum
   2. volume
   3. sentiment
   4. open interest
   5. data across markets
   6. etc
4. If you're using two or more trading indicators, they shall be of different types *"two momentum indicators aren't better then one"*
5. Bollinger bands can be used to identify / elucidate price patterns like *M* on top or *W* on bottom
6. The mere crossing of the prices with band's limits does not indicate that you should buy or sell anything (at least not by itself - again, **use more indicators**)
7. In markets of high tendency (upward or downward trend) the prices will **cross both** top and bottom bands
8. Market close outside the bands are **continuation** indicators, not **reversion**
9. The use of $20$ days for the calculation are not complete standards, it may work on the majority of the shares but better results can be archived by adapting this value to the specific stock / market that you're operating on
10. The crossing of the prices with the central line does not indicate a **strong** new trend, it's a "**medium**" confirmation that a new trend is coming
11. If you decide to change the amount of days used on the $sma$ you should also change the $stdev$ multiplication factor (lets call it $\alpha$)
    * **if** $d=20$ **then** $\alpha$ should be $2$
    * **if** $d=50$ **then** $\alpha$ should be $2.1$
    * **if** $d=10$ **then** $\alpha$ should be $1.9$
    * *Note: the book that I'm consulting to write this information does not make clear what rule should i use for any value of $d$* ![](https://i.ibb.co/SxvbjNg/Screenshot-20200213-123109.png) From regressions calculated on Geogebra, considering only this three examples we can arrive in some formula between
    * $\alpha(days)=0.0046 * days + 1.8791$ (*linear*) **to** $\alpha(days) = 1.88 \ * \ 1^days$ (*fitting growth*) 
12. The traditional Bolling bands uses a **simple moving average** to be logically consistent because that's what is used to calculate the **standard deviation**
13. You can use the **exponential moving average** in the calculations - this will eliminate sudden changes in the band width caused by a large price changes - this average shall be used for the moving average and the standard deviation on all lines
14. Do not make statistical assumptions based on the standard deviation - the distribution is **not normal** and the  $n$ of samples aren't big enough for having **statistical significance** (you could expect $90\%$ instead of $95\%$ of the data inside the band with the default parameters)
15. Bollinger bands does not provide a continuous signalization - instead they help to identify configurations where the probabilities may be in your favor



In [28]:
import sys
import os

import chart_studio.plotly as plty
import plotly.graph_objs as gobjs

import plotly

sys.path.insert(0, os.path.abspath('../py'))
from Secrets import ReadSecrets
secrets = ReadSecrets().read('../secrets.json')
plty.sign_in(secrets.secrets['plotly']['username'], secrets.secrets['plotly']['apiKey'])

from datetime import datetime as calendar
import numpy as np

from pandas_datareader import data as pdr
import pandas as pd
import yfinance as yf
yf.pdr_override()

from DataHelper import DataHelper

from plotly import tools

In [29]:
data, close, extractors = DataHelper.get_history_formatted(['AAPL', 'MSFT', '^GSPC'], calendar(2016, 1 ,1), calendar(2020, 2, 1))
msft = extractors['MSFT']()
aapl = extractors['AAPL']()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


## With SMA

In [30]:
window = 20
alpha = 0.0046 * window + 1.8791

print(f'Considering a window of {window} days, the alpha (sdev multiplication factor) was calculated as {alpha}')

Considering a window of 20 days, the alpha (sdev multiplication factor) was calculated as 1.9711


In [31]:
aapl_avg = aapl.Close.rolling(window=window).mean().dropna()
aapl_std = aapl.Close.rolling(window=window).std().dropna()

In [32]:
aapl_bollinger = pd.DataFrame(index=aapl_avg.index)
aapl_bollinger['mband'] = aapl_avg
aapl_bollinger['uband'] = aapl_avg + aapl_std.apply(lambda x: (x * alpha))
aapl_bollinger['lband'] = aapl_avg - aapl_std.apply(lambda x: (x * alpha))

In [33]:
aapl_prices = aapl[aapl_bollinger.index[0]:]

upper_band = gobjs.Scatter(
                x=aapl_bollinger.index,
                y=aapl_bollinger.uband,
                name = "Upper Band",
                line = dict(color = '#17BECF'),
                opacity = 1,
            )


lower_band = gobjs.Scatter(
                x=aapl_bollinger.index,
                y=aapl_bollinger.lband,
                name = "Lower Band",
                line = dict(color = '#17BECF'),
                opacity = 1,
                fill='tonexty'
            )

ma = gobjs.Scatter(
                x=aapl_bollinger.index,
                y=aapl_bollinger.mband,
                name = "Moving average",
                line = dict(color = '#B22222'),
                opacity = 1
            )

prices = gobjs.Candlestick(
                x=aapl_prices.index,
                open=aapl_prices.Open,
                high=aapl_prices.High,
                low=aapl_prices.Low,
                name = "Prices",
                close=aapl_prices.Close
            )





data = [upper_band, lower_band, prices, ma]
plty.iplot(data)

In [37]:
variation = gobjs.Scatter(
                x=aapl_bollinger.index,
                y=aapl_bollinger.uband - aapl_bollinger.lband,
                name = "Band width",
                line = dict(color = '#17BECF'),
                opacity = 1,
            )

data = [variation]
plty.iplot(data)

## With EMA

In [27]:
aapl_ewm_avg = aapl.Close.ewm(span=window,adjust=False).mean().dropna()
aapl_ewm_std = aapl.Close.ewm(span=window,adjust=False).std().dropna()

In [34]:
aapl_bollinger_ewma = pd.DataFrame(index=aapl_avg.index)
aapl_bollinger_ewma['mband'] = aapl_ewm_avg
aapl_bollinger_ewma['uband'] = aapl_ewm_avg + aapl_ewm_std.apply(lambda x: (x * alpha))
aapl_bollinger_ewma['lband'] = aapl_ewm_avg - aapl_ewm_std.apply(lambda x: (x * alpha))

In [35]:
aapl_prices = aapl[aapl_bollinger_ewma.index[0]:]

upper_band = gobjs.Scatter(
                x=aapl_bollinger_ewma.index,
                y=aapl_bollinger_ewma.uband,
                name = "Upper Band",
                line = dict(color = '#17BECF'),
                opacity = 1,
            )


lower_band = gobjs.Scatter(
                x=aapl_bollinger_ewma.index,
                y=aapl_bollinger_ewma.lband,
                name = "Lower Band",
                line = dict(color = '#17BECF'),
                opacity = 1,
                fill='tonexty'
            )

ma = gobjs.Scatter(
                x=aapl_bollinger_ewma.index,
                y=aapl_bollinger_ewma.mband,
                name = "Moving average",
                line = dict(color = '#B22222'),
                opacity = 1
            )

prices = gobjs.Candlestick(
                x=aapl_prices.index,
                open=aapl_prices.Open,
                high=aapl_prices.High,
                low=aapl_prices.Low,
                name = "Prices",
                close=aapl_prices.Close
            )





data = [upper_band, lower_band, prices, ma]
plty.iplot(data)

In [42]:
variation = gobjs.Scatter(
                x=aapl_bollinger_ewma.index,
                y=aapl_bollinger_ewma.uband - aapl_bollinger.lband,
                name = "Band width",
                line = dict(color = '#17BECF'),
                opacity = 1,
            )

average = gobjs.Scatter(
                x=aapl_bollinger_ewma.index,
                y=(aapl_bollinger_ewma.uband - aapl_bollinger.lband).ewm(span=30,adjust=False).mean(),
                name = "Average",
                line = dict(color = '#DD9999'),
                opacity = 1,
            )


data = [variation, average]
plty.iplot(data)